In [1]:
import sys, time, math, os

import tensorflow as tf
import numpy as np

from reg import run_reg
from audio_batch import get_valid
from audio_batch import batch_audio
from util import weight_variable
from reg_nn import gen_relu_layer, gen_layer

-------

In [2]:
sin_pre = np.load('./data/lowpass/pre/sin.npz')['data']
sqr_pre = np.load('./data/lowpass/pre/square.npz')['data']
saw_pre = np.load('./data/lowpass/pre/plysaw.npz')['data']

sin_post = np.load('./data/lowpass/post/sin.npz')['data']
sqr_post = np.load('./data/lowpass/post/square.npz')['data']
saw_post = np.load('./data/lowpass/post/plysaw.npz')['data']


set_in = np.concatenate([sin_pre[0], sin_pre[1], sqr_pre[0], sqr_pre[1], saw_pre[0], saw_pre[1]])
set_out = np.concatenate([sin_post[0], sin_post[1], sqr_post[0], sqr_post[1], saw_post[0], saw_post[1]])

train_in, train_out, valid_in, valid_out = get_valid(set_in, set_out, 1, .25)

if not train_in.shape[0] + valid_in.shape[0] == train_out.shape[0] + valid_out.shape[0] == set_in.shape[0] == set_out.shape[0]:
    raise ValueError('audio shapes don\'t match up')

input_set, output_set = batch_audio(train_in, train_out, .5, offset=.1)
valid_in_batches  = valid_in.reshape(int(valid_in.shape[0] / input_set.shape[1]), input_set.shape[1])
valid_out_batches = valid_out.reshape(int(valid_out.shape[0] / output_set.shape[1]), output_set.shape[1])

train_ref_std = output_set.std()

# input_set = input_set.reshape(-1, input_set.shape[1], 1)
# output_set = output_set.reshape(-1, output_set.shape[1], 1)
# valid_in_batches = valid_in_batches.reshape(-1, valid_in_batches.shape[1], 1)
# valid_out_batches = valid_out_batches.reshape(-1, valid_out_batches.shape[1], 1)

In [3]:
def gen_lin(layer_width):
    std = 0.1
    alpha = 0.00001

    ### GEN LAYERS
    x = tf.placeholder(tf.float32, shape=[None, input_set.shape[1]], name='x')
    y = tf.placeholder(tf.float32, shape=[None, output_set.shape[1]], name='y')

    w0 = tf.Variable(tf.truncated_normal([input_set.shape[1], layer_width], stddev=std), name='w0')
    b0 = tf.Variable(tf.truncated_normal([1,layer_width], stddev=std), name='b0')
    lay0 = tf.matmul(x,w0) + b0

    w1 = tf.Variable(tf.truncated_normal([layer_width, layer_width], stddev=std), name='w1')
    b1 = tf.Variable(tf.truncated_normal([1,layer_width], stddev=std), name='b1')
    lay1 = tf.matmul(lay0,w1) + b1
    
    w2 = tf.Variable(tf.truncated_normal([layer_width, output_set.shape[1]], stddev=std), name='w2')
    b2 = tf.Variable(tf.truncated_normal([1,output_set.shape[1]], stddev=std), name='b2')
    lay2 = tf.matmul(lay1,w2) + b2

    P = lay2

    MSE = tf.reduce_mean(tf.square(P - y))
    L2 = alpha * (tf.nn.l2_loss(w0) + tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))

    optimizer = tf.train.AdamOptimizer().minimize(MSE + L2)
    
    global_step = tf.Variable(0, name='global_step', trainable=False)

    saver = tf.train.Saver(
        { "w0": w0,
          "b0": b0,
          "w1": w1,
          "b1": b1,
          "w2": w2,
          "b2": b2,
          "global_step": global_step})

    return x, y, MSE, P, optimizer, global_step, saver

In [4]:
def header():
    print('\t                         mse                    rmse                                 std            ')
    print('\t        training  validation    training  validation    training  validation   reference     runtime')

In [8]:
def run_lin(hidden_width, epochs):
    x, y, MSE, P, optimizer, global_step, saver = gen_lin(hidden_width)

    sess = tf.Session()
    sess.run(tf.initialize_all_variables())
    
    ckpt_dir = "./tmp/lowpass/linear/%d/" % hidden_width
    
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    else:
        ckpt = tf.train.get_checkpoint_state(ckpt_dir)
        if ckpt and ckpt.model_checkpoint_path:
            print('restoring network from:',ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)

    batch_size = 500
    epoch = sess.run(global_step)
    t_start = time.time()
    total_compute_time = -1
    
    check_dist = int((epochs / 100))

    print("starting from epoch:", epoch)

    header()
    while (epoch < epochs):
        perm = np.random.permutation(input_set.shape[0])

        start = 0
        for k in range( math.ceil( input_set.shape[0] / batch_size ) ):
            batch = perm[ start:start + batch_size ]
            sess.run([optimizer],feed_dict={x:input_set[batch],y:output_set[batch]})
            start += batch_size
        print('.', end="", flush=True)

        epoch+=1
        sess.run(global_step.assign(epoch))

        if epoch % check_dist == 0:
            save_path = saver.save(sess, ckpt_dir + 'model.ckpt')
            (mse_train, p_train) = sess.run([MSE, P],feed_dict={x:input_set,y:output_set})
            (mse_valid, p_valid) = sess.run([MSE, P],feed_dict={x:valid_in_batches,y:valid_out_batches})
            train_std = (output_set - p_train).std()
            valid_std = (valid_out_batches - p_valid).std()

            total_compute_time = (time.time() - t_start)/60
            print()
            print('epoch:%5d %12.5f%12.5f%12.5f%12.5f%12.5f%12.5f%12.5f%12.1f' % (epoch, mse_train, mse_valid, np.sqrt(mse_train), np.sqrt(mse_valid), train_std, valid_std, train_ref_std, total_compute_time), end=" ")


    # compute final results (and ensure computed if we're already done)
    (mse_train, p_train) = sess.run([MSE, P],feed_dict={x:input_set,y:output_set})
    (mse_valid, p_valid) = sess.run([MSE, P],feed_dict={x:valid_in_batches,y:valid_out_batches})
    train_std = (output_set - p_train).std()
    valid_std = (valid_out_batches - p_valid).std()

    print()
    header()
    print('epoch:%5d %12.5f%12.5f%12.5f%12.5f%12.5f%12.5f%12.5f%12.1f' % (epoch, mse_train, mse_valid, np.sqrt(mse_train), np.sqrt(mse_valid), train_std, valid_std, train_ref_std, total_compute_time))

In [ ]:
run_lin(2000, 4000)

restoring network from: ./tmp/lowpass/linear/2000/model.ckpt
starting from epoch: 2227
	                         mse                    rmse                                 std            
	        training  validation    training  validation    training  validation   reference     runtime
.............
epoch: 2240      0.00325     0.00561     0.05698     0.07491     0.05722     0.07493     0.11060         1.3 ................

--------------

In [2]:
sin_pre = np.load('./data/lowpass/pre/sin.npz')['data']
sqr_pre = np.load('./data/lowpass/pre/square.npz')['data']
saw_pre = np.load('./data/lowpass/pre/plysaw.npz')['data']

sin_post = np.load('./data/lowpass/post/sin.npz')['data']
sqr_post = np.load('./data/lowpass/post/square.npz')['data']
saw_post = np.load('./data/lowpass/post/plysaw.npz')['data']


set_in = np.concatenate([sin_pre[0], sin_pre[1], sqr_pre[0], sqr_pre[1], saw_pre[0], saw_pre[1]])
set_out = np.concatenate([sin_post[0], sin_post[1], sqr_post[0], sqr_post[1], saw_post[0], saw_post[1]])

train_in, train_out, valid_in, valid_out = get_valid(set_in, set_out, 1, .25)

if not train_in.shape[0] + valid_in.shape[0] == train_out.shape[0] + valid_out.shape[0] == set_in.shape[0] == set_out.shape[0]:
    raise ValueError('audio shapes don\'t match up')

input_set, output_set = batch_audio(train_in, train_out, .5, offset=.1)
valid_in_batches  = valid_in.reshape(int(valid_in.shape[0] / input_set.shape[1]), input_set.shape[1])
valid_out_batches = valid_out.reshape(int(valid_out.shape[0] / output_set.shape[1]), output_set.shape[1])

input_set = input_set.reshape(-1, input_set.shape[1], 1)
output_set = output_set.reshape(-1, output_set.shape[1], 1)
valid_in_batches = valid_in_batches.reshape(-1, valid_in_batches.shape[1], 1)
valid_out_batches = valid_out_batches.reshape(-1, valid_out_batches.shape[1], 1)

In [3]:
height_in = input_set.shape[1]
height_out = output_set.shape[1]
std = 0.1
alpha = 0.00001

x = tf.placeholder(tf.float32, shape=[None, height_in, 1])
y = tf.placeholder(tf.float32, shape=[None, height_out, 1])

layers = []


filter_1 = weight_variable([500, 1, 500])
b_conv_1 = weight_variable([int(filter_1.get_shape()[2])], std=std)
conv_1 = tf.nn.conv1d(x, filter_1, stride=1, padding="SAME")

layers.append([tf.nn.relu(conv_1 + b_conv_1), tf.nn.l2_loss(conv_1)])

# filter_2 = weight_variable([7, 500, 1000])
# b_conv_2 = weight_variable([1,int(filter_2.get_shape()[2])], std=std)

# print(layers[-1][0])
# conv_2 = tf.nn.conv1d(layers[-1][0], filter_2, stride=1, padding="SAME")

# layers.append([tf.nn.relu(conv_2 + b_conv_2), tf.nn.l2_loss(conv_2)])

# reshaped = tf.reshape(layers[-1][0], [-1, height_in * 500]
# layers.append(gen_layer(reshaped), int(height_in * 500), height_out, std))

# layers = np.asarray(layers)

# MSE = tf.reduce_mean(tf.square(layers[-1][0] - y))
# L2 = alpha * (layers[:,1].sum())

# optimizer = tf.train.AdamOptimizer().minimize(MSE + L2)

# P = layers[-1][0]

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [88]:
epochs = 100
t_start = time.time()

train_ref = output_set.mean()
valid_ref = valid_out_batches.mean()
train_ref_std = (output_set - train_ref).std()

print('                               mse                                 std            ')
print('    epoch     training  validation    training  validation   reference     runtime')

epoch = 0
for _ in range(epochs):
    perm = np.random.permutation(input_set.shape[0])
    sess.run([optimizer],feed_dict={x:input_set[perm],y:output_set[perm]})

    if epoch % (epochs / 100) == 0:
        (mse_train, p_train) = sess.run([MSE, P],feed_dict={x:input_set,y:output_set})
        (mse_valid, p_valid) = sess.run([MSE, P],feed_dict={x:valid_in_batches,y:valid_out_batches})
        train_std = (output_set - p_train).std()
        valid_std = (valid_out_batches - p_valid).std()

        total_compute_time = (time.time() - t_start)/60
        print('      %3d %12.5f%12.5f%12.5f%12.5f%12.5f%12.1f' % (epoch, mse_train, mse_valid, train_std, valid_std, train_ref_std, total_compute_time))
    epoch+=1

print('\t                       mse                                 std            ')
print('\t      training  validation    training  validation   reference     runtime')
print('epoch:%3d %12.5f%12.5f%12.5f%12.5f%12.5f%12.1f' % (epoch, mse_train, mse_valid, train_std, valid_std, train_ref_std, total_compute_time))

                               mse                                 std            
    epoch     training  validation    training  validation   reference     runtime


InvalidArgumentError: Input to reshape is a tensor with 159104 values, but the requested shape requires a multiple of 22050
	 [[Node: Reshape_22 = Reshape[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](Relu_41, Reshape_22/shape)]]
Caused by op 'Reshape_22', defined at:
  File "/opt/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/opt/anaconda3/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).s

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))